In [1]:
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize

**<span style="font-size:2em;">Clean and Process on Firm Name Data (MOVE TO NEW SCRIPT and save clean firms as a file)</span>**

**All of the Firm Name Data**

In [2]:
# # Firm name data 
# df_firms_crsp = pd.read_csv('../../Data/FirmNameData/firm.names.crsp.csv')
# df_firms_earn = pd.read_csv('../../Data/FirmNameData/firm.names.earnings.wsh.csv')
# df_firms_execucomp = pd.read_csv('../../Data/FirmNameData/firm.names.execucomp.csv')
# df_firms_crsp.columns, df_firms_earn.columns, df_firms_execucomp.columns = ['Name'],['Name'],['Name']

# # Firm name data from wiki (different format)
# df_firms_wiki = pd.read_csv('../../Data/FirmNameData/firm.names.wiki.csv')
# df_firms_wiki.columns = ['Name', 'Ticker', 'Market', 'Website', 'Exchange']
# df_firms_wiki_names = pd.DataFrame(df_firms_wiki.Name)

# # Condense into one dataframe for ease of searching (size 21335)
# df_firms = ((df_firms_crsp.append(df_firms_earn)).append(df_firms_execucomp)).append(df_firms_wiki_names)

# # Remove duplicate entries
# firms_with_duplicates = [firm.lower() for firm in df_firms['Name'].tolist()]
# firms_no_duplicates = []
# for firm in firms_with_duplicates:
#     if firm not in firms_no_duplicates:
#         firms_no_duplicates.append(firm)

**CRSP Firm Name Data (For Testing)**

In [3]:
df_firms_crsp = pd.read_csv('../../Data/FirmNameData/firm.names.crsp.csv')
df_firms_crsp.columns = ['Name']

**Read State Abreviations Data for Cleaning (Have not used this yet)**

In [4]:
df_states = pd.read_excel('../../Data/Other/state_abv.xlsx')
states_abv = [str(ab).lower() for ab in df_states['USPS Abbreviation'].tolist()]

**Function to Join Single Characters in Name**

In [5]:
def join_name(row):
    split = row.split()
    new_string = []
    single_chars = []
    rest_of_string = []
    for i in range(len(split)):
            if len(split[i]) == 1 and not split[i].isnumeric():
                single_chars.append((split[i].lower(), i))
            else:
                rest_of_string.append((split[i].lower(), i))
                
    if len(single_chars) <= 1:
        return row.lower()
    else:
        chars_to_remove = []
        # Edge cases
        for i in range(len(single_chars)):
            # Lone single char is first letter. Example: 'X ENERGY GROUP'
            if i == 0:
                if single_chars[i][1] != single_chars[i + 1][1] - 1:
                    chars_to_remove.append(single_chars[i])
            # Lone single char is last letter. Example: 'XC ENERGY GROUP H'
            elif i == len(single_chars) - 1:
                if single_chars[i][1] != single_chars[i - 1][1] + 1: 
                    chars_to_remove.append(single_chars[i])
            # Lone single char is between two words letter. Example: 'XC ENERGY H GROUP'
            else:
                if single_chars[i][1] != single_chars[i - 1][1] + 1 and single_chars[i][1] != single_chars[i + 1][1] - 1:
                    chars_to_remove.append(single_chars[i])

        for char in chars_to_remove:
            if char in single_chars:
                single_chars.remove(char)
            rest_of_string.append(char)

        while len(single_chars) > 0:
            j = 0
            tent_l = []
            while j < len(single_chars) - 1 and single_chars[j][1] == single_chars[j + 1][1] - 1:
                if single_chars[j] not in tent_l:
                    tent_l.append(single_chars[j])
                if single_chars[j + 1] not in tent_l: 
                    tent_l.append(single_chars[j + 1])
                j += 1

            for letter in tent_l:
                if letter in single_chars:
                    single_chars.remove(letter)

            rest_of_string.append((''.join([tpl[0] for tpl in tent_l]), tent_l[0][1]))

        rest_of_string.sort(key = lambda x: x[1]) 
        return ' '.join([tpl[0] for tpl in rest_of_string])

**Function to Remove Words from Firm Names**

In [7]:
def remove_words(list_of_words, row):
    tokens = word_tokenize(row)
    new_name = []
    for word in tokens:
        if word not in list_of_words:
            new_name.append(word)
    return ' '.join(new_name)

**Join Single Characters**

In [8]:
df_firms_crsp['Cleaned_Name'] = df_firms_crsp['Name'].apply(lambda name: join_name(name))

**Remove words like 'Inc', 'Corp', 'Co's, and state abreviations From Firm Names**

In [9]:
words_to_remove = ['inc', 'corp', 'co', 'mlp'] + states_abv
df_firms_crsp['Cleaned_Name'] = df_firms_crsp['Cleaned_Name'].apply(lambda name: remove_words(words_to_remove, name))

**Save to New File**

In [10]:
df_firms_crsp.to_excel('../../Data/FirmNameData/cleaned_firms.xlsx', index=False)